In [1]:
#hide
! [ -e /content ] & pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#hide
from fastbook import *

In [3]:
key = os.environ.get('AZURE_SEARCH_KEY', '7ff0ce7725b44de4a0b1279abff69e2a')

In [4]:
players = 'Neymar','Cristiano Ronaldo','Messi'
path = Path('player')

In [5]:
if not path.exists():
    path.mkdir()
    for o in players:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, o)
        download_images(dest, urls=results.attrgot('contentUrl'))

In [6]:
fns = get_image_files(path)
fns

(#427) [Path('player/Messi/7bfeeea5-188c-4c27-9221-c41fb67a7fc9.jpg'),Path('player/Messi/28c904f7-e3c1-4a00-ae43-ed42b6beb566.jpg'),Path('player/Messi/3ee52b88-f1e2-4ef1-944a-70e6a7ff897e.jpg'),Path('player/Messi/7a98b536-683c-42aa-9009-e019a2ae59cd.jpg'),Path('player/Messi/d7cd6736-0841-43a3-804f-4709a028769b.jpg'),Path('player/Messi/bb320395-9ecf-4a0e-b415-8e863016f09b.jpg'),Path('player/Messi/4457f2a7-a637-437e-83fa-bc0086b27c71.jpeg'),Path('player/Messi/15b48f06-d221-4d0a-8440-f79fa9f0836f.jpg'),Path('player/Messi/e5f18cbe-6a45-4033-ab6a-69a61951dc55.jpg'),Path('player/Messi/6d2af7a6-bfc0-4f61-8fc0-f98da5d8d3a2.jpg')...]

In [7]:
failed = verify_images(fns)
failed

(#10) [Path('player/Messi/f611dcfd-0ea2-4e9e-b68b-bfb8307b0b39.jpg'),Path('player/Messi/1fb3eb36-c28b-40cf-a5ae-5905ce63c774.jpg'),Path('player/Messi/97faeaac-14a4-4749-b371-2d48b4999c95.jpg'),Path('player/Messi/39ada810-f831-401c-a39c-b0b6c25ef77c.png'),Path('player/Messi/948c0ca5-5d73-4ec9-ab06-ba31f596a61d.jpg'),Path('player/Cristiano Ronaldo/088dcb22-1e29-4c42-8dde-1f4aca3075a0.jpg'),Path('player/Cristiano Ronaldo/6d833bfe-e545-444b-976a-05b09239a46c.jpg'),Path('player/Cristiano Ronaldo/3141b774-3edc-412e-97a2-fc918564a51a.jpg'),Path('player/Neymar/d1951413-46d3-464b-a2b0-29c3593e76eb.jpg'),Path('player/Neymar/6bfe3b19-21a4-45c8-b737-2dd40dc62f68.jpg')]

In [8]:
failed.map(Path.unlink);

In [9]:
player = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [10]:
player = player.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = player.dataloaders(path)

In [11]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.0MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,1.628635,1.419876,0.614458,01:52


/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


epoch,train_loss,valid_loss,error_rate,time
0,1.198767,0.929419,0.421687,02:14
1,1.130082,0.754737,0.325301,02:11
2,0.945361,0.647891,0.277108,02:16
3,0.822350,0.629513,0.289157,02:13


/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should b

In [12]:
learn.export()